In [5]:
import numpy as np, os
import pandas as pd

# import optuna

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717 
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

In [6]:
import random    
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4069).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4069), (6182,), (1545, 10, 4069), (1545,))

In [17]:
checkpoint_filepath = f"test.hdf5"

lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
es = EarlyStopping(monitor="val_loss", patience=5, verbose=1, mode="min", restore_best_weights=True)
sv = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)
        
history = lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep, callbacks=[lr, es, sv], shuffle=False)

results = []
print(' Computing LSTM feature importance...')

# COMPUTE BASELINE (NO SHUFFLE)
oof_preds = lstm.predict(X_test, verbose=0)
baseline_mse = np.square(np.subtract(oof_preds,y_test)).mean()
results.append({'feature':'BASELINE','baseline_mse':baseline_mse}) 

Epoch 1/500
37/37 [==============================] - 5s 119ms/step - loss: 0.3664 - acc: 0.8557 - val_loss: 0.5126 - val_acc: 0.7697

Epoch 00001: val_loss improved from inf to 0.51263, saving model to test.hdf5
Epoch 2/500
37/37 [==============================] - 4s 111ms/step - loss: 0.3316 - acc: 0.8790 - val_loss: 0.5358 - val_acc: 0.7749

Epoch 00002: val_loss did not improve from 0.51263
Epoch 3/500
37/37 [==============================] - 4s 109ms/step - loss: 0.3102 - acc: 0.8932 - val_loss: 0.5593 - val_acc: 0.7743

Epoch 00003: val_loss did not improve from 0.51263
Epoch 4/500
37/37 [==============================] - 4s 108ms/step - loss: 0.2843 - acc: 0.9053 - val_loss: 0.5894 - val_acc: 0.7665

Epoch 00004: val_loss did not improve from 0.51263
Epoch 5/500
37/37 [==============================] - 4s 110ms/step - loss: 0.2693 - acc: 0.9154 - val_loss: 0.6047 - val_acc: 0.7704

Epoch 00005: val_loss did not improve from 0.51263
Epoch 6/500
37/37 [=============================

In [ ]:
gpu_strategy = tf.distribute.get_strategy()

with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):

        # SHUFFLE FEATURE K
        save_col = X_test[:,:,k].copy()
        np.random.shuffle(X_test[:,:,k])

        # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
        oof_preds = lstm.predict(X_test, verbose=0)
        mse = np.square(np.subtract(oof_preds,y_test)).mean()

        results.append({'feature':COLS[k],'mse':mse})
        X_test[:,:,k] = save_col
        # DISPLAY LSTM FEATURE IMPORTANCE
    #     print()
        df = pd.DataFrame(results)
        df = df.sort_values('mse')

  0%|          | 0/4069 [00:00<?, ?it/s]

# binary_crossentropy

In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy

lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
es = EarlyStopping(monitor="val_loss", patience=5, verbose=1, mode="min", restore_best_weights=True)
        
history = lstm.fit(X_train, y_train, validation_split=0.25, batch_size=128, epochs=ep, callbacks=[lr, es], shuffle=False)

results = []
print(' Computing LSTM feature importance...')

# COMPUTE BASELINE (NO SHUFFLE)
from tensorflow.keras.losses import BinaryCrossentropy
oof_preds = lstm.predict(X_test, verbose=0)
baseline_bce = BinaryCrossentropy(y_test, oof_preds)
results.append({'feature':'BASELINE','baseline_bce':baseline_bce}) 

gpu_strategy = tf.distribute.get_strategy()

with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):

        # SHUFFLE FEATURE K
        save_col = X_test[:,:,k].copy()
        np.random.shuffle(X_test[:,:,k])

        # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
        oof_preds = lstm.predict(X_test, verbose=0)
        bce = BinaryCrossentropy(y_test, oof_preds)

        results.append({'feature':COLS[k],'mse':bce})
        X_test[:,:,k] = save_col
        # DISPLAY LSTM FEATURE IMPORTANCE
    #     print()
        df = pd.DataFrame(results)
        df = df.sort_values('bce')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *

seed_num = 42
tf.random.set_seed(seed_num)

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1, activation='sigmoid'))
ep = 500
pa = 30
early_stop = EarlyStopping(monitor='val_acc', patience=pa, verbose=1, restore_best_weights=True)
lstm.compile(optimizer= "adam", loss = "binary_crossentropy", metrics=['acc'])

results = []

# COLS
a = pd.read_csv('total_data.csv')
COLS = list(a['ITEMID'].sort_values().unique())

gpu_strategy = tf.distribute.get_strategy()

with gpu_strategy.scope():
    for k in tqdm(range(len(COLS))):

        # SHUFFLE FEATURE K
        save_col = X_test[:,:,k].copy()
        np.random.shuffle(X_test[:,:,k])

        # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
        oof_preds = lstm.predict(X_test, verbose=0)
        mae = np.mean(np.abs( oof_preds-y_test ))

        results.append({'feature':COLS[k],'mae':mae})
        X_test[:,:,k] = save_col
        
        # DISPLAY LSTM FEATURE IMPORTANCE
        df = pd.DataFrame(results)
        df = df.sort_values('mae')

In [ ]:
# 실행이 잘 안됨
EPOCH = 300
BATCH_SIZE = 1024

gpu_strategy = tf.distribute.get_strategy()

with gpu_strategy.scope():
    test_preds = []
    model = keras.models.Sequential([
        keras.layers.InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])),
        keras.layers.LSTM(128, activation='hard_sigmoid', return_sequences=True),
        keras.layers.LSTM(64, activation='hard_sigmoid', return_sequences=True),
        keras.layers.Dropout(0.2),
        keras.layers.LSTM(64, activation='hard_sigmoid', return_sequences=True),
        keras.layers.LSTM(32, activation='hard_sigmoid', return_sequences=False),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='sigmoid')])
    model.compile(optimizer="adam", loss="binary_crossentropy")

    lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
    es = EarlyStopping(monitor="val_loss", patience=5, verbose=1, mode="min", restore_best_weights=True)
    sv = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
        save_weights_only=False, mode='auto', save_freq='epoch',
        options=None
    )
    model.fit(X_train, y_train, validation_split=0.25,epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, es, sv])


    print(' Predicting test data...')
    test_preds.append(model.predict(X_test,verbose=0).reshape(-1, 1))

    results = []
    print(' Computing LSTM feature importance...')

    # COMPUTE BASELINE (NO SHUFFLE)
    oof_preds = model.predict(X_test, verbose=0)  
    baseline_mae = np.mean(np.abs( oof_preds-y_test ))
    results.append({'feature':'BASELINE','mae':baseline_mae})           

    for k in tqdm(range(len(COLS))):

        # SHUFFLE FEATURE K
        save_col = X_test[:,:,k].copy()
        np.random.shuffle(X_test[:,:,k])

        # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
        oof_preds = model.predict(X_test, verbose=0)
        mae = np.mean(np.abs( oof_preds-y_test ))
        results.append({'feature':COLS[k],'mae':mae})
        X_test[:,:,k] = save_col

        # SAVE LSTM FEATURE IMPORTANCE
        df = df.sort_values('mae',ascending=False)
        df.to_csv(f'lstm_feature_importance_fold_{fold+1}.csv',index=False)